<a href="https://colab.research.google.com/github/ckgpeace/EVA5B2/blob/main/S4/Session4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
        
#         self.conv1 = nn.Conv2d(1, 32, 3, padding=1)     # inp-28 out-28 RF-
#         self.conv2 = nn.Conv2d(32, 64, 3, padding=1)    # 28, 28
#         self.pool1 = nn.MaxPool2d(2, 2)                 # 28, 14

#         self.conv3 = nn.Conv2d(64, 128, 3, padding=1)   # 14, 14
#         self.conv4 = nn.Conv2d(128, 256, 3, padding=1)  # 14, 14
#         self.pool2 = nn.MaxPool2d(2, 2)                 # 14, 7    

#         self.conv5 = nn.Conv2d(256, 512, 3)             # 7, 5
#         self.conv6 = nn.Conv2d(512, 1024, 3)            # 5, 3    
#         self.conv7 = nn.Conv2d(1024, 10, 3)             # 3, 1    
#         self.gap = nn.AvgPool2d(1)

#     def forward(self, x):
#         x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
#         x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
#         x = F.relu(self.conv6(F.relu(self.conv5(x))))
#         x = self.conv7(x)
#         x = self.gap(x)
#         x = x.view(-1, 10)
#         return F.log_softmax(x)


In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Sequential(
        nn.Conv2d(1, 32, 3,  padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(32),

        nn.Conv2d(32, 64, 3, padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(64),
        nn.MaxPool2d(2, 2),
        nn.Dropout(0.25)
        ) # in = 28, out = 14, RF = 10


        self.conv2 = nn.Sequential(
        nn.Conv2d(64, 128, 3,  padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(128),

        nn.Conv2d(128, 256, 3, padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(256),
        nn.MaxPool2d(2, 2),
        nn.Dropout(0.25)
        ) # in = 14, out = 7, RF = 28


        self.conv3 = nn.Sequential(
        nn.Conv2d(256, 256, 3,  padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(256),

        nn.Conv2d(256, 10, 3, padding=1)
        ) # in = 7, out = 7, RF = 32

        self.gap = nn.AvgPool2d(7) 
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.gap(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)



In [4]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
              ReLU-2           [-1, 32, 28, 28]               0
       BatchNorm2d-3           [-1, 32, 28, 28]              64
            Conv2d-4           [-1, 64, 28, 28]          18,496
              ReLU-5           [-1, 64, 28, 28]               0
       BatchNorm2d-6           [-1, 64, 28, 28]             128
         MaxPool2d-7           [-1, 64, 14, 14]               0
           Dropout-8           [-1, 64, 14, 14]               0
            Conv2d-9          [-1, 128, 14, 14]          73,856
             ReLU-10          [-1, 128, 14, 14]               0
      BatchNorm2d-11          [-1, 128, 14, 14]             256
           Conv2d-12          [-1, 256, 14, 14]         295,168
             ReLU-13          [-1, 256, 14, 14]               0
      BatchNorm2d-14          [-1, 256,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [5]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [6]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [7]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 10):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0487, Accuracy: 9853/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0292, Accuracy: 9906/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0161, Accuracy: 9951/10000 (100%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0164, Accuracy: 9948/10000 (99%)



loss=0.012447402812540531 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 28.80it/s]



Test set: Average loss: 0.0149, Accuracy: 9951/10000 (100%)

